In [10]:
import numpy as np
from ldpc import bposd_decoder
from ldpc.codes import rep_code 
from bposd.hgp import hgp
from tqdm import tqdm
import time
from qecsim import paulitools as pt 
from qecsim.models.generic import DepolarizingErrorModel
from qecsim.models.toric import ToricCode
import galois
from typing import List, Tuple
import nbimporter
import BP_OSD_Simulation_V2 as my_bposd

In [6]:
dim = 6
my_code = ToricCode(dim,dim)
my_error_model = DepolarizingErrorModel()
GF = galois.GF(2)
num_qubits = 2 * dim ** 2
error_probability = 0.08
z_stabs = (my_code.stabilizers[:dim ** 2])[:, 2 * dim ** 2:]
x_stabs = (my_code.stabilizers[dim ** 2:])[:, :2 * dim ** 2]

In [7]:
import numpy as np
from ldpc import bposd_decoder

bpd_X=bposd_decoder(
    z_stabs,#the parity check matrix
    error_rate=error_probability,
    channel_probs=[None], #assign error_rate to each qubit. This will override "error_rate" input variable
    max_iter=num_qubits, #the maximum number of iterations for BP)
    bp_method="ms",
    ms_scaling_factor=0, #min sum scaling factor. If set to zero the variable scaling factor method is used
    osd_method="osd_cs", #the OSD method. Choose from:  1) "osd_e", "osd_cs", "osd0"
    osd_order=7 #the osd search depth
    )

bpd_Z=bposd_decoder(
    x_stabs,#the parity check matrix
    error_rate=error_probability,
    channel_probs=[None], #assign error_rate to each qubit. This will override "error_rate" input variable
    max_iter=num_qubits, #the maximum number of iterations for BP)
    bp_method="ms",
    ms_scaling_factor=0, #min sum scaling factor. If set to zero the variable scaling factor method is used
    osd_method="osd_cs", #the OSD method. Choose from:  1) "osd_e", "osd_cs", "osd0"
    osd_order=7 #the osd search depth
    )

In [8]:
rng = np.random.default_rng()
# Error: random error based on error probability
error = my_error_model.generate(my_code, error_probability, rng)
while (sum(error) == 0):
    error = my_error_model.generate(my_code, error_probability, rng)

z_error = error[2 * dim**2:]
x_error = error[:2 * dim**2]
z_syndrome = np.mod(x_stabs @ z_error,2)
x_syndrome = np.mod(z_stabs @ x_error, 2)
bpd_X.decode(x_syndrome)
bpd_Z.decode(z_syndrome)

print("X Error")
print(x_error)
print("Z Error")
print(z_error)
print("BP+OSD Decoding X")
print(bpd_X.osdw_decoding)
print("BP+OSD Decoding Z")
print(bpd_Z.osdw_decoding)
#Decoding is successful if the residual error commutes with the logical operators
recovery_x = (bpd_X.osdw_decoding) %2
recovery_z = (bpd_Z.osdw_decoding) %2
residual_error_x = (recovery_x + x_error) %2
residual_error_z = (recovery_z + z_error) %2

print("Commutation with Stabs")
print(np.sum((z_stabs@residual_error_x) % 2))
print(np.sum((x_stabs@residual_error_z) % 2))

X Error
[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0
 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
Z Error
[0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
BP+OSD Decoding X
[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0
 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
BP+OSD Decoding Z
[0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 1 0 0 0 1 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
Commutation with Stabs
0
0


In [14]:
num_rounds = 2000
codespace_projection = [0]*num_rounds
logical_errors = [0]*num_rounds
uncorrected_logical_error_rate = [0]*num_rounds
count = 0

for r in tqdm(range(num_rounds)):
    start_time = time.time()
    rng = np.random.default_rng()
    # Error: random error based on error probability
    error = my_error_model.generate(my_code, error_probability, rng)
    while (sum(error) == 0):
        error = my_error_model.generate(my_code, error_probability, rng)

    z_error = error[2 * dim**2:]
    x_error = error[:2 * dim**2]
    z_syndrome = np.mod(x_stabs @ z_error,2)
    x_syndrome = np.mod(z_stabs @ x_error, 2)
    bpd_X.decode(x_syndrome)
    bpd_Z.decode(z_syndrome)

    recovery = np.array(np.hstack((bpd_X.osdw_decoding, bpd_Z.osdw_decoding)), dtype='int32')
    runtime = time.time() - start_time

    if sum(pt.bsp(recovery ^ error, my_code.stabilizers.T)) > 0:
        codespace_projection[r] = 1
        f.write(error_string)
    if sum(pt.bsp(recovery ^ error, my_code.logicals.T)) > 0:
        logical_errors[r] = 1
    if sum(pt.bsp(error, my_code.logicals.T)) > 0:
        uncorrected_logical_error_rate[r] = 1
    

100%|██████████| 2000/2000 [00:00<00:00, 3390.00it/s]


In [15]:
print('number of codespace projection failures', sum(codespace_projection))
print('logical error rate', sum(logical_errors)/len(logical_errors))
print('uncorrected error rate', sum(uncorrected_logical_error_rate)/len(uncorrected_logical_error_rate))
print('physical error rate', error_probability)

number of codespace projection failures 0
logical error rate 0.0815
uncorrected error rate 0.6725
physical error rate 0.08
